In [28]:
import json
import pandas as pd
import re

In [29]:
from synthetic_invoice_generator import *

In [30]:
def json_to_df(filename):
    '''
    converts json string from json file to pd dataframe
    '''
    with open(filename) as file:
        data = json.load(file)

    formatted_data = []
    for item in data:
        row = {entry["field"]: entry["value"] for entry in item["extractions"]}
        formatted_data.append(row)

    df = pd.DataFrame(formatted_data)
    return df

In [31]:
def string_to_df(json_string):
    '''
    converts json string to pd dataframe
    '''
    formatted_data = []
    for item in json_string:
        row = {entry["field"]: entry["value"] for entry in item["extractions"]}
        formatted_data.append(row)

    df = pd.DataFrame(formatted_data)
    return df

In [32]:
data = generate_dataset(num_invoices=1000,num_merchants=10000,anomaly_rate=0.4, seed=42)
df = string_to_df(data)
df

Generating 1000 synthetic invoices across a set of 10000 merchants (anomaly rate: 40.0%)...
Analysis of 1000 invoices:
- Unique merchants: 914
- Total line items: 3462
- Average line items per invoice: 3.46

Field frequency:
- merchant: 1000 (100.0%)
- invoice_date: 1000 (100.0%)
- merchant_branch: 1000 (100.0%)
- merchant_chain: 1000 (100.0%)
- due_date: 1000 (100.0%)
- payment_terms: 1000 (100.0%)
- grand_total: 1000 (100.0%)
- tax: 1000 (100.0%)
- po_number: 1000 (100.0%)
- merchant_address: 1000 (100.0%)
- payment_method: 1000 (100.0%)
- country: 1000 (100.0%)
- currency: 1000 (100.0%)
- line_details: 1000 (100.0%)
Generated 1000 synthetic invoices and saved to synthetic_invoices.json
Saved invoices in JSONL format to synthetic_invoices.jsonl


,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,payment_method,country,currency,line_details
0,Barry,02/08/2025,Barry,Barry,02/08/2025,DUE ON RECEIPT,14331.45,821.64,K29420,141 Hodges Mountains Kramerville NM 04216 USA,FedEx Express,US,USD,"[{'line_count': '1', 'line_description': 'NN-0..."
1,Macdonald-Patterson Office Supplies International,02/06/2025,Macdonald-Patterson Office Supplies International,Macdonald-Patterson Office Supplies International,02/06/2025,DUE ON RECEIPT,2553.65,94.63,E51230,57081 Wilkinson Cove East David OR 03013 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'DDR-..."
2,Sawyer Laboratory,02/22/2025,Sawyer Laboratory,Sawyer Laboratory,02/22/2025,DUE ON RECEIPT,3637.04,44.34,474017,193 Ryan Lodge Suite 052 West Sylviachester NC...,PayPal,US,USD,"[{'line_count': '1', 'line_description': 'K-38..."
3,Stark-Church Enterprises,03/01/2025,Stark-Church Enterprises,Stark-Church Enterprises,03/31/2025,NET 30 DAYS,96408.56,5636.28,PO-86574,413 James Roads Bakerville IA 12699 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'M-80..."
4,Bond Corp.,12/23/2024,Bond Corp.,Bond Corp.,01/28/2025,NET 45 DAYS,8167.53,490.36,CUST-70023-2402,746 Michael Trafficway Michelleburgh NV 02384 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'MLF-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Moore Construction Ltd.,08/29/2024,Moore Construction Ltd.,Moore Construction Ltd.,08/29/2024,COD,48643.69,5671.09,706201,10862 Donovan Lights Apt. 433 Port Johnathan N...,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'UW-3..."
996,IndustryAssume Electronics,01/31/2025,IndustryAssume Electronics,IndustryAssume Electronics,03/02/2025,NET 30 DAYS,3829.68,216.77,881190,44333 Mark Roads New Patrick DC 31035 USA,Check,US,USD,"[{'line_count': '1', 'line_description': 'B-84..."
997,KidEnergy Aerospace Ltd.,11/01/2024,KidEnergy Aerospace Ltd.,KidEnergy Aerospace Ltd.,12/01/2024,NET 30 DAYS,12752.72,0.00,U72327,4951 Joshua Ports South Timothy PW 89016 USA,DHL Express,US,USD,"[{'line_count': '1', 'line_description': 'D-04..."
998,WillRepresent Inc.,02/20/2025,WillRepresent Inc.,WillRepresent Inc.,03/22/2025,2/10 NET 30,554.91,11.01,2503-5957,29098 Amy Ville South Jeremymouth NY 15844 USA,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'GR-9..."


In [33]:
# Convert data types
df["grand_total"] = df["grand_total"].astype(float)
df["tax"] = df["tax"].astype(float)
df["due_date"] = pd.to_datetime(df["due_date"], errors="coerce")
df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")

# Derived features
df["cal_invoice_age"] = (df["due_date"] - df["invoice_date"]).dt.days
df["num_line_items"] = df["line_details"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df["avg_line_price"] = df["grand_total"] / df["num_line_items"].replace(0, np.nan)
df["avg_line_price"] = df["avg_line_price"].round(2)

df

,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,payment_method,country,currency,line_details,cal_invoice_age,num_line_items,avg_line_price
0,Barry,2025-02-08,Barry,Barry,2025-02-08,DUE ON RECEIPT,14331.45,821.64,K29420,141 Hodges Mountains Kramerville NM 04216 USA,FedEx Express,US,USD,"[{'line_count': '1', 'line_description': 'NN-0...",0,3,4777.15
1,Macdonald-Patterson Office Supplies International,2025-02-06,Macdonald-Patterson Office Supplies International,Macdonald-Patterson Office Supplies International,2025-02-06,DUE ON RECEIPT,2553.65,94.63,E51230,57081 Wilkinson Cove East David OR 03013 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'DDR-...",0,4,638.41
2,Sawyer Laboratory,2025-02-22,Sawyer Laboratory,Sawyer Laboratory,2025-02-22,DUE ON RECEIPT,3637.04,44.34,474017,193 Ryan Lodge Suite 052 West Sylviachester NC...,PayPal,US,USD,"[{'line_count': '1', 'line_description': 'K-38...",0,2,1818.52
3,Stark-Church Enterprises,2025-03-01,Stark-Church Enterprises,Stark-Church Enterprises,2025-03-31,NET 30 DAYS,96408.56,5636.28,PO-86574,413 James Roads Bakerville IA 12699 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'M-80...",30,4,24102.14
4,Bond Corp.,2024-12-23,Bond Corp.,Bond Corp.,2025-01-28,NET 45 DAYS,8167.53,490.36,CUST-70023-2402,746 Michael Trafficway Michelleburgh NV 02384 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'MLF-...",36,5,1633.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Moore Construction Ltd.,2024-08-29,Moore Construction Ltd.,Moore Construction Ltd.,2024-08-29,COD,48643.69,5671.09,706201,10862 Donovan Lights Apt. 433 Port Johnathan N...,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'UW-3...",0,6,8107.28
996,IndustryAssume Electronics,2025-01-31,IndustryAssume Electronics,IndustryAssume Electronics,2025-03-02,NET 30 DAYS,3829.68,216.77,881190,44333 Mark Roads New Patrick DC 31035 USA,Check,US,USD,"[{'line_count': '1', 'line_description': 'B-84...",30,3,1276.56
997,KidEnergy Aerospace Ltd.,2024-11-01,KidEnergy Aerospace Ltd.,KidEnergy Aerospace Ltd.,2024-12-01,NET 30 DAYS,12752.72,0.00,U72327,4951 Joshua Ports South Timothy PW 89016 USA,DHL Express,US,USD,"[{'line_count': '1', 'line_description': 'D-04...",30,2,6376.36
998,WillRepresent Inc.,2025-02-20,WillRepresent Inc.,WillRepresent Inc.,2025-03-22,2/10 NET 30,554.91,11.01,2503-5957,29098 Amy Ville South Jeremymouth NY 15844 USA,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'GR-9...",30,2,277.46


In [37]:
payment_days = {
    "NET 30 DAYS": 30,
    "NET 45 DAYS": 45,
    "NET 60 DAYS": 60,
    "NET 15 DAYS": 15,
    "DUE ON RECEIPT": 0,
    "COD": 0,
    "2/10 NET 30": 30 
}

df["expected_invoice_age"] = df["payment_terms"].map(payment_days)
df["invoice_age_mismatch"] = (df["expected_invoice_age"] != df["cal_invoice_age"]).astype(int)

df[df["invoice_age_mismatch"] == 1]

,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,payment_method,country,currency,line_details,cal_invoice_age,num_line_items,avg_line_price,expected_invoice_age,invoice_age_mismatch
4,Bond Corp.,2024-12-23,Bond Corp.,Bond Corp.,2025-01-28,NET 45 DAYS,8167.53,490.36,CUST-70023-2402,746 Michael Trafficway Michelleburgh NV 02384 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'MLF-...",36,5,1633.51,45,1
7,Weaver Electronics,2024-07-29,Weaver Electronics,Weaver Electronics,2024-08-20,NET 30 DAYS,5122.47,417.95,11898-277,0427 Jefferson Motorway Suite 884 North Wendyp...,ACH,US,USD,"[{'line_count': '1', 'line_description': 'B-84...",22,3,1707.49,30,1
11,White-Schwartz Telecommunications,2025-02-16,White-Schwartz Telecommunications,White-Schwartz Telecommunications,2025-03-11,2/10 NET 30,53926.97,0.00,2503-9726,56761 Donald Fields Susanmouth OR 26003 USA,DHL Express,US,USD,"[{'line_count': '1', 'line_description': 'GBL-...",23,4,13481.74,30,1
14,Robertson & Allen Medical,2025-02-01,Robertson & Allen Medical,Robertson & Allen Medical,2025-03-28,NET 45 DAYS,126454.54,7157.80,2503-5313,858 Holmes Grove East Julieview PA 08219 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'NN-0...",55,2,63227.27,45,1
22,Knight & Jackson Telecommunications,2025-01-25,Knight & Jackson Telecommunications,Knight & Jackson Telecommunications,2025-03-12,NET 60 DAYS,2241.60,94.67,CUST-74497-8745,96241 Calderon Islands North Lindseymouth OK 9...,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'RGJ-...",46,1,2241.60,60,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Church-Hansen,2024-11-30,Church-Hansen,Church-Hansen,2025-02-13,NET 60 DAYS,8913.05,0.00,CUST-74497-7745,088 David Rest Thomasmouth MT 47899 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'C-03...",75,3,2971.02,60,1
986,Campaign LLC,2025-03-12,Campaign LLC,Campaign LLC,2025-04-16,NET 30 DAYS,9427.68,401.54,88462-787,715 Lang Shore Apt. 674 Yvetteton MO 93484 USA,PayPal,US,USD,"[{'line_count': '1', 'line_description': 'X-84...",35,4,2356.92,30,1
988,Mclean,2025-03-07,Mclean,Mclean,2025-05-03,NET 45 DAYS,2025.18,77.89,U89242,3414 Tammy Pine West Clifford AL 18649 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'GU-6...",57,1,2025.18,45,1
994,WallShow,2024-10-05,WallShow,WallShow,2024-11-12,NET 30 DAYS,871.70,47.26,CUST-50895-1746,61422 Joseph Circles New Isaiah UT 83940 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'B-84...",38,4,217.92,30,1


In [38]:
with open("synthetic_data_configs.json", "r") as f:
    config = json.load(f)
state_tax_rates = config["tax_rates"]

# Function to extract state from address (format: "Street, City, State ZIP, Country")
def extract_state(address):
    match = re.search(r'\b[A-Z]{2}\b', address)  # Match two-letter state codes
    return match.group(0) if match else None

# Apply state extraction
df["state"] = df["merchant_address"].apply(extract_state)

# Map expected tax rate based on extracted state
df["expected_tax_rate"] = df["state"].map(state_tax_rates)

# State has expected_tax_rate == NaN
df["state_mismatch_flag"] = df["expected_tax_rate"].isna().astype(int)

def compute_subtotal(line_details):
    return sum(float(line["line_total"]) for line in line_details)

df["sub_total"] = df["line_details"].apply(compute_subtotal)
df["cal_tax_rate"] = df["tax"] / df["sub_total"]
df["tax_mismatch_flag"] = (df["cal_tax_rate"].round(4) != df["expected_tax_rate"].round(4)).astype(int)

df

,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,...,num_line_items,avg_line_price,expected_invoice_age,invoice_age_mismatch,state,expected_tax_rate,state_mismatch_flag,sub_total,cal_tax_rate,tax_mismatch_flag
0,Barry,2025-02-08,Barry,Barry,2025-02-08,DUE ON RECEIPT,14331.45,821.64,K29420,141 Hodges Mountains Kramerville NM 04216 USA,...,3,4777.15,0,0,NM,0.0513,0,13509.81,0.060818,1
1,Macdonald-Patterson Office Supplies International,2025-02-06,Macdonald-Patterson Office Supplies International,Macdonald-Patterson Office Supplies International,2025-02-06,DUE ON RECEIPT,2553.65,94.63,E51230,57081 Wilkinson Cove East David OR 03013 USA,...,4,638.41,0,0,OR,0.0000,0,2459.02,0.038483,1
2,Sawyer Laboratory,2025-02-22,Sawyer Laboratory,Sawyer Laboratory,2025-02-22,DUE ON RECEIPT,3637.04,44.34,474017,193 Ryan Lodge Suite 052 West Sylviachester NC...,...,2,1818.52,0,0,NC,0.0475,0,3592.70,0.012342,1
3,Stark-Church Enterprises,2025-03-01,Stark-Church Enterprises,Stark-Church Enterprises,2025-03-31,NET 30 DAYS,96408.56,5636.28,PO-86574,413 James Roads Bakerville IA 12699 USA,...,4,24102.14,30,0,IA,0.0600,0,90772.28,0.062093,1
4,Bond Corp.,2024-12-23,Bond Corp.,Bond Corp.,2025-01-28,NET 45 DAYS,8167.53,490.36,CUST-70023-2402,746 Michael Trafficway Michelleburgh NV 02384 USA,...,5,1633.51,45,1,NV,0.0685,0,7677.17,0.063872,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Moore Construction Ltd.,2024-08-29,Moore Construction Ltd.,Moore Construction Ltd.,2024-08-29,COD,48643.69,5671.09,706201,10862 Donovan Lights Apt. 433 Port Johnathan N...,...,6,8107.28,0,0,NH,0.0000,0,42934.52,0.132087,1
996,IndustryAssume Electronics,2025-01-31,IndustryAssume Electronics,IndustryAssume Electronics,2025-03-02,NET 30 DAYS,3829.68,216.77,881190,44333 Mark Roads New Patrick DC 31035 USA,...,3,1276.56,30,0,DC,0.0600,0,3612.91,0.059999,0
997,KidEnergy Aerospace Ltd.,2024-11-01,KidEnergy Aerospace Ltd.,KidEnergy Aerospace Ltd.,2024-12-01,NET 30 DAYS,12752.72,0.00,U72327,4951 Joshua Ports South Timothy PW 89016 USA,...,2,6376.36,30,0,PW,NaN,1,12713.11,0.000000,1
998,WillRepresent Inc.,2025-02-20,WillRepresent Inc.,WillRepresent Inc.,2025-03-22,2/10 NET 30,554.91,11.01,2503-5957,29098 Amy Ville South Jeremymouth NY 15844 USA,...,2,277.46,30,0,NY,0.0400,0,543.90,0.020243,1
